# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
cities_weather = pd.read_csv("../Output/weather.csv")
cities_weather.head()

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Country,Wind Speed,Date
0,Khatanga,71.97,102.50,3.94,92,0,RU,8.34,1603315999
1,Ostrovnoy,68.05,39.51,29.84,97,93,RU,7.40,1603316095
2,Busselton,-33.65,115.33,48.00,75,1,AU,10.56,1603316095
3,Luganville,-15.53,167.17,78.80,88,75,VU,6.98,1603316096
4,Bambous Virieux,-20.34,57.76,75.20,78,75,MU,9.17,1603316096


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
locations = cities_weather[["Latitude", "Longitude"]]
map_fig = gmaps.figure()
humidity = cities_weather["Humidity"]
heat_layer = gmaps.heatmap_layer(locations, weights=humidity)
map_fig.add_layer(heat_layer)
map_fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [21]:
# Condition: between 60-70 degrees F
good_temp = cities_weather.loc[cities_weather["Max Temperature"]>60]
good_temp = good_temp.loc[good_temp["Max Temperature"]<70]
good_temp = good_temp.reset_index()
good_temp.head()

,index,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Country,Wind Speed,Date
0,11,Tiznit Province,29.58,-9.50,64.71,74,99,MA,2.82,1603316097
1,14,Albany,42.60,-73.97,69.01,80,22,US,5.01,1603315865
2,15,Flinders,-34.58,150.86,63.00,90,86,AU,1.01,1603316097
3,18,Coffs Harbour,-30.30,153.13,68.00,72,90,AU,6.93,1603316098
4,20,Cidreira,-30.18,-50.21,66.38,89,100,BR,15.59,1603316098


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [29]:
hotels = []

print("Beginning Hotel List")
print('-------------------------')

for city in range(len(good_temp)):
    lat = good_temp.loc[city]['Latitude']
    lng = good_temp.loc[city]['Longitude']
    
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(url, params=params).json()
    
    try:
        hotels.append(response['results'][0]['name'])
        print(f"Appending {response['results'][0]['name']}")
    except:
        hotels.append("")
        print("Could not find hotel. Skipping....")

Beginning Hotel List
-------------------------
Appending Ouijjane
Appending Voorheesville
Appending Wollongong
Appending Coffs Harbour
Appending CIDREIRA
Appending Algeciras
Appending Horta
Appending Paris
Appending Port Alfred
Appending Akçakoca
Appending Port Elizabeth
Appending Ribeira Brava
Appending Isiro
Appending Pisco
Appending Hentiesbaai
Appending Obo
Appending Carnarvon
Appending Zhanjiang
Appending Ajdabiya
Appending Karachi
Appending Santa Cruz Da Graciosa
Appending Coyhaique
Appending Lompoc
Appending Cincinnati
Appending Lüneburg
Appending Valdivia
Appending Oranjemund
Appending Najran
Appending Taltal
Appending Maralal
Appending Castro
Appending Fortuna
Appending Harnai
Appending Katsuura
Appending Villacarrillo
Appending Tocra
Appending Tacoronte
Appending Koblenz
Appending Sydney
Appending Lima
Appending Atsabe
Appending Comodoro Rivadavia
Appending Anamur
Appending Tura
Appending Riyadh
Appending Luderitz
Appending Plettenberg Bay
Appending Cleveland
Appending Huacho

In [32]:
good_temp["Hotel Name"] = hotels
good_temp = good_temp.dropna(how='any')
good_temp.head()

,index,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Country,Wind Speed,Date,Hotel Name
0,11,Tiznit Province,29.58,-9.50,64.71,74,99,MA,2.82,1603316097,Ouijjane
1,14,Albany,42.60,-73.97,69.01,80,22,US,5.01,1603315865,Voorheesville
2,15,Flinders,-34.58,150.86,63.00,90,86,AU,1.01,1603316097,Wollongong
3,18,Coffs Harbour,-30.30,153.13,68.00,72,90,AU,6.93,1603316098,Coffs Harbour
4,20,Cidreira,-30.18,-50.21,66.38,89,100,BR,15.59,1603316098,CIDREIRA


In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
